# Model Evaluation: Mel Cepstral Distortion with Dynamic Time Warping

In this tutorial, we will learn how to calculate **mel cepstral distortion (MCD)** with **dynamic time warping (DTW)** between a synthesized mel spec and a reference mel spec. MCD DTW can be useful for comparing models trained on the same training data.

MCD is an objective measure of speech quality that is calculated between pairs of TTS-generated mel spectrograms and ground truth mel spectrograms. Two mel spectrograms are similar if the MCD value between them is low; as you might expect, the MCD of a mel spec with itself is 0.

MCD DTW is a modification of MCD that works with non-aligned mels by using a dynamic time warping cost matrix. (Vanilla MCD can only be measured for two mels that are both the same length, and which are assumed to be aligned.) The scale depends on factors such as the mel extractor and reduction algorithm (mean of DTW cost or min DTW path cost).

## License

> Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.
>
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
>     http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
BRANCH = 'r1.22.0'
# If you're using Google Colab and not running locally, uncomment and run this cell.
# !pip install librosa numpy matplotlib

In [ ]:
## Import libraries
import IPython.display as ipd
import librosa
import numpy as np
import math
import matplotlib.pyplot as plt
import os

## Defining Parameters for Spectrogram Generation

In this section we define parameters required to generate our mel spectrograms. More information about these parameters can found in the Librosa documentation for [stft](https://librosa.org/doc/main/generated/librosa.stft.html), [mels](https://librosa.org/doc/main/generated/librosa.filters.mel.html) and [mfcc](https://librosa.org/doc/main/generated/librosa.feature.mfcc.html) generation.

In [ ]:
## Mel spec params
n_fft=1024
hop_length=256
win_length=None
window='hann'
n_mels = 80
sr = 22050

## Mfcc params
n_mfcc=34

A little bit about these parameters:  
    - `n_fft`: Number of fft_components or length of windowed signal after padding in STFT.  
    - `hop_length`: Number of audio samples between adjacent STFT columns.  
    - `window`: Window to use in STFT.  
    - `win_length`: Length of the window to be used.  
    - `n_mels`: Number of number of mel bands to generate.  
    - `sr`: Sample rate of the samples.  
    - `n_mfcc`: Number of MFCCs to generate.

## Loading and Visualizing Data

Lets apply the algorithm on a synthesized mel and ground truth mel pair for understanding.

First, we need a function to generate mel spectrograms from audio files. Mel spectrograms are generated using the [librosa mel extractor](https://librosa.org/doc/main/generated/librosa.filters.mel.html)

In [ ]:
def wav2mel(filename):
    """
    Function to load an audio file and generate/return mel specs:
    Args:
        filename: Full path of the audio file.
    Returns:
        mels: Corresponding mel spectrogram.
    """
    wav_, _ = librosa.load(filename, sr=sr)  # load() returns an (audio data, sample rate) tuple
    mels = librosa.feature.melspectrogram(
        y=wav_,
        sr=sr,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=win_length,
        window=window,
        n_mels=n_mels
    )
    mels = librosa.power_to_db(mels, ref=np.max)
    return mels

We also need a function to convert mels to MFCC, we will use Librosa's [mfcc](https://librosa.org/doc/main/generated/librosa.feature.mfcc.html) generation:

In [ ]:
def mel2mfcc(mels):
    mfcc = librosa.feature.mfcc(S=mels, n_mfcc=n_mfcc)
    return mfcc

For this tutorial, we have already generated mels from trained [FastPitch](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/tts_en_fastpitch) and [RAD-TTS](https://github.com/NVIDIA/radtts) models, and we have the ground truth sample that they correspond to. Let's first download the tarball with the data and expand it.

In [ ]:
# Untar the example files.
!wget https://tts-tutorial-data.s3.us-east-2.amazonaws.com/MCD_DTW_examples.tar
!tar -xvf MCD_DTW_examples.tar

Now we can generate a mel spectrogram for the ground truth audio, load mel specs for generated audio and generate MFCC(Mel frequency cepstral coefficient) for both.

In [ ]:
## Generate spectrograms
gt_mels = wav2mel("MCD_DTW/gt/sample_0.wav")
synt_mels = np.load("MCD_DTW/fastpitch/mels/mels_0.npy")

## Generate MFCCs
gt_mfcc = mel2mfcc(gt_mels)
synt_mfcc = mel2mfcc(synt_mels)

### Visualizing the Audio

Let's first listen to the ground truth and synthesized audio.

In [ ]:
print("Ground truth audio:")
audio_gt, sr_gt = librosa.load("MCD_DTW/gt/sample_0.wav")
ipd.display(ipd.Audio(audio_gt, rate=sr_gt))

print("Synthesized audio:")
audio_fp, sr_fp = librosa.load("MCD_DTW/fastpitch/audio/sample_0.wav")
ipd.display(ipd.Audio(audio_fp, rate=sr_fp))

Now, let's take a look at the mel spectrograms.

In [ ]:
# Visualize mel spectrograms
fig, ax = plt.subplots(2, 1, figsize=(12,10))

_ = ax[0].pcolormesh(gt_mels, cmap='viridis')
_ = ax[0].set_title("Ground Truth Mel Spectrogram")

_ = ax[1].pcolormesh(synt_mels, cmap='viridis')
_ = ax[1].set_title("FastPitch Synthesized Mel Spectrogram")

## Calculating the DTW matrix

Dynamic time warping finds the most optimal path to align two sequences of different length, and in doing so measures the similarity between the two time series that are not in sync.

DTW uses dynamic programming to **calculate the cost of every alignment path and chooses the path with least accumulated cost**. The value of accumulated cost matrix $D$ at index $(x_a, y_b)$ is the minimum distance between the points, where $x$ and $y$ are two time series. Formally, the cost matrix can be defined as:
  
$D(a,b) = min(D(a-1, b), D(a, b-1), D(a-1, b-1)) + c(x_{a}, y_{b})$  
$D(1, b) = \sum(c(1, y_{b}))$  
$D(a, 1) = \sum(c(x_{a}, 1))$ 
  
Where ***x*** and ***y*** are the audio signals and ***c*** is the log cost function we have defined.

We will use [the DTW function from Librosa](https://librosa.org/doc/main/generated/librosa.sequence.dtw.html) on MFCC. It returns the DTW accumulated cost matrix and DTW optimum path.

In the following function, we define the cost function for DTW to pass in.

In [ ]:
## Define the cost function for calculating DTW
def log_spec_dB_dist(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))

Now we can run Librosa's `dtw()`:

In [ ]:
# dtw_cost: Shape (N, M), where N and M are the lengths of gt_mfcc, synt_mfcc respectively. Cost matrix.
# dtw_min_path: Shape (N, 2). Pairs of coordinates with of the min-cost path from bottom-right to top-left.
dtw_cost, dtw_min_path = librosa.sequence.dtw(gt_mfcc, synt_mfcc, metric=log_spec_dB_dist)

Reduction of the DTW matrix can be done by either taking the mean of the entire cost matrix or averaging the DTW cost for the minimum cost path per frame. We will use the DTW cost along the min cost path.

In [ ]:
# First, sum up the costs over the path
path_cost_matrix = dtw_cost[dtw_min_path[:, 0], dtw_min_path[:, 1]]
path_cost = np.sum(path_cost_matrix)

# Average over path length
path_length = dtw_min_path.shape[0]
reduced_dtw_cost = path_cost/path_length

# Average over number of frames
frames = synt_mels.shape[1]
mcd = reduced_dtw_cost/frames

print(f"MCD_DTW is: {mcd}")

## MCD DTW Usecase

MCD is a very useful metric to **compare the convergence of two models**. Therefore in this section, we will calculate the average MCD for audio files generated by two different models: FastPitch and RAD-TTS.

But first, let's put the calculations we have just performed in functions for better readability and reusability.

Here are the helper functions for getting the average cost of DTW along a path.

In [ ]:
def extract_path_cost(D, wp):
    """
    Get the path cost from D(cost matrix), wp (warped path)
    Returns the sum of the path cost 
    """
    path_cost = D[wp[:, 0], wp[:, 1]]
    return np.sum(path_cost)

def extract_frame_avg_path_cost(D, wp):
    """
    Get the average path cost over the length of the given path
    """
    path_cost = extract_path_cost(D, wp)
    path_length = wp.shape[0]
    frame_avg_path_cost = path_cost / float(path_length)
    return frame_avg_path_cost

And a function for calculating MCD for a synthetic mel spectrogram:

In [ ]:
def cal_mcd(gt_mfcc, synt_mfcc, cost_function, dtw_type='path_cost'):
    """
    Calculates MCD between a ground truth and synthetic mel.
    """
    frames = synt_mfcc.shape[1]
    path_cost = 0
    
    # dynamic time warping for MCD
    dtw_cost, dtw_min_path = librosa.sequence.dtw(gt_mfcc, synt_mfcc, metric=cost_function)
    if dtw_type == 'mean':
        path_cost = np.mean(dtw_cost)
    else:
        path_cost = extract_frame_avg_path_cost(dtw_cost, dtw_min_path)
    
    mcd = path_cost / frames
    
    return mcd, frames

This last function will calculate the MCDs for all the synthetic mels in a directory.

We'll get a better comparison if we compute the MCDs of (much) more than one synthesized spectrogram per model! This function streamlines the process by letting us pass in (1) a directory with all the synthesized mel spectrograms from a TTS model, and (2) a directory with all the ground-truth audio.

The function assumes that, after sorting the contents of each directory, wav *#i* in the ground truth directory will correspond to mel *#i* in the synthesized mel directory. In our case, the directories for ground truth and synthesized FastPitch mels look like this:

```bash
% ls gt/
sample_0.wav  sample_1.wav  sample_2.wav  sample_3.wav  sample_4.wav
% ls fastpitch/mels/
mels_0.npy  mels_1.npy  mels_2.npy  mels_3.npy  mels_4.npy
```

We only have ten files each to compare as a toy example, for a full evaluation you will likely want more!

In [ ]:
def cal_mcd_dir(synt_dir, gt_dir):
    """
    Calculate MCDs for pairs of synthetic and ground truth audio files.
    
    Args:
        synt_dir: Path to the directory that contains all the synthetic mels.
        gt_dir: Path to the directory that contains all the ground truth wavs.
            These should correspond 1:1 with the mels in synt_dir.
            
    Returns:
        List of MCDs (where length is the number of files in each directory)
    """
    mcds = []
    
    synt_filenames = os.listdir(synt_dir)
    synt_filepaths = [os.path.join(synt_dir, filename) for filename in synt_filenames]
    synt_filepaths.sort()
    gt_filenames = os.listdir(gt_dir)
    gt_filepaths = [os.path.join(gt_dir, filename) for filename in gt_filenames]
    gt_filepaths.sort()

    for synt_melname, gt_audio in zip(synt_filepaths, gt_filepaths):
        synt_mels = np.load(synt_melname)
        synt_mfcc = mel2mfcc(synt_mels)
        
        gt_mels = wav2mel(gt_audio)
        gt_mfcc = mel2mfcc(gt_mels)
        
        mcd, _ = cal_mcd(gt_mfcc, synt_mfcc, log_spec_dB_dist)
        mcds.append(mcd)
    return mcds

### Calculate MCD DTW on Synthesized Files from Each Model

We can now calculate the MCD DTW for the synthesized FastPitch mels (compared to the ground truth) as well as for the synthesized RAD-TTS mels (ditto), then compare them. This will take a few seconds.

In [ ]:
%%capture --no-display
mels_dir_m1 = "MCD_DTW/fastpitch/mels/"
mels_dir_m2 = "MCD_DTW/radtts/mels/"
mels_dir_gt = "MCD_DTW/gt/"

mcds_m1 = cal_mcd_dir(mels_dir_m1, mels_dir_gt)  # FastPitch
mcds_m2 = cal_mcd_dir(mels_dir_m2, mels_dir_gt)  # RAD-TTS

In [ ]:
print(f"Average MCD for Model 1 (FastPitch) is: {sum(mcds_m1)/len(mcds_m1):.2f}")
print(f"Average MCD for Model 2 (RAD-TTS) is:   {sum(mcds_m2)/len(mcds_m2):.2f}")

We're measuring divergence from the ground truth for each of these, so **lower is better**!

### Plotting MCD

We can also plot the MCD DTW values for both models, with the model with a lower MCD DTW value being closer to ground truth.

In [ ]:
x_axis = np.linspace(1, len(mcds_m1), len(mcds_m1)) ## Define an x axis for for plotting in matplotlib
plt.plot(x_axis, mcds_m1, label="FastPitch")
plt.plot(x_axis, mcds_m2, label="RAD-TTS")

plt.title("MCD DTW value for each file")
plt.ylabel("MCD_DTW value")

plt.grid()
plt.legend()
plt.show()

From the graph above, we can see that in general the value of MCD is greater for RAD-TTS mel spectrograms than for the FastPitch ones. This is also reflected in the average MCD value for both models that we computed before.

Therefore, we can conclude that FastPitch has better convergence than RAD-TTS. However, we cannot evaluate the quality of audio generated by these models using MCD alone! **MCD is a great tool for testing model convergence, but generated audio may have pronunciation and quality artifacts.** Therefore MCD evaluation should be followed by a MOS (Mean Opinion Score) and CMOS (Comparative Mean Opinion Score) evaluation.

## Additional NeMo Resources

If you are unsure where to begin for training a TTS model, you may want to start with the [FastPitch and Mixer-TTS Training notebook](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/FastPitch_MixerTTS_Training.ipynb) or the [NeMo TTS Primer notebook](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/NeMo_TTS_Primer.ipynb). For fine-tuning, there is also the [FastPitch Fine-Tuning notebook](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/FastPitch_Finetuning.ipynb).

For some guidance on how to load a trained model and perform inference to generate mels or waveforms, check out how it's done in the [Inference notebook](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/Inference_ModelSelect.ipynb). Important functions to know are include `from_pretrained()` (if loading from an NGC model) and `restore_from()` (if loading a `.nemo` file). See the [NeMo Primer notebook](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/00_NeMo_Primer.ipynb) for more general information about model training, saving, and loading.